In [11]:
import pandas as pd

In [12]:
# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [13]:
# Calculate the total number of schools
total_schools = school_data["School ID"].count()

#Calculate the total number of students
total_students = student_data["Student ID"].count()

# Calculate sum of budgets
total_budget = school_data["budget"].sum()

#Calculate average math scores
math_average = student_data["math_score"].mean()

#Calculate average reading scores
read_average = student_data["reading_score"].mean()

# math_pass = number of students that have a math score >= 70
math_pass_df = school_data_complete.loc[(school_data_complete['math_score'] >= 70)]
math_pass = math_pass_df["Student ID"].count()

# read_pass = number of students that have a read score >= 70
read_pass_df = school_data_complete.loc[(school_data_complete['reading_score'] >= 70)]
read_pass = read_pass_df["Student ID"].count()

# pass = number of students that have a math AND reading score >= 70
pass_df = school_data_complete.loc[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70), :]
pass_grade = pass_df["Student ID"].count()

# percentage of students with passing math grade
per_math_pass = (math_pass/total_students) * 100

# percentage of students with passing reading grade
per_read_pass = (read_pass/total_students) * 100

# percentage of students with both passing reading and math grades
per_pass = (pass_grade/total_students) * 100

total_schools, total_students, total_budget, math_average, read_average, math_pass, read_pass, per_pass

(15,
 39170,
 24649428,
 78.98537145774827,
 81.87784018381414,
 29370,
 33610,
 65.17232575950983)

In [14]:
#create District Summary data frame
district_summary = pd.DataFrame([
    {"Total Schools": total_schools,
    "Total Students": total_students,
    "Total Budget": total_budget,
    "Average Math Score": math_average,
    "Average Reading Schore": read_average,
    "% Passing Math": per_math_pass,
    "% Passing Reading": per_read_pass,
    "% Overall Passing": per_pass,}
])
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Schore,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [15]:
# Extract the columns with numerical values
school_scores = school_data_complete[["school_name", "reading_score", "math_score", "size"]]

#group by school and calculate the mean
school_group = school_scores.groupby(["school_name"])

avg_scores = school_group.mean()

# Using .rename(columns={}) in order to rename columns
renamed_avg_scores = avg_scores.rename(columns={"reading_score":"Reading Average", "math_score":"Math Average", "size":"Total Students"})

# Filter rows so that only rows with passing math scores are shown
passing_math_scores = school_scores.loc[(school_scores['math_score'] >= 70)]

# Group by school and count rows
passing_math_scores_group = passing_math_scores.groupby(["school_name"])

num_pass_math = passing_math_scores_group.count()

# Drop unnecessary columns
drop_read = num_pass_math.drop('reading_score', 1)
new_num_pass_math = drop_read.drop('size', 1)

# Filter rows so that only rows with passing reading scores are shown
passing_read_scores = school_scores.loc[(school_scores['reading_score'] >= 70)]

# Group by school and count rows
passing_read_scores_group = passing_read_scores.groupby(["school_name"])

num_pass_read = passing_read_scores_group.count()

# Drop unnecessary columns
drop_math = num_pass_read.drop('math_score', 1)
new_num_pass_read = drop_math.drop('size', 1)

# Filter rows so that only rows with passing reading and math scores are shown
passing_both_scores = school_scores.loc[(school_scores['reading_score'] >= 70) & (school_scores['math_score'] >= 70)]
passing_both_scores

# Group by school and count rows
passing_both_scores_group = passing_both_scores.groupby(["school_name"])

num_pass_both = passing_both_scores_group.count()

# Drop unnecessary columns
drop_it = num_pass_both.drop('reading_score', 1)
new_num_pass_both = drop_it.drop('math_score', 1)

# Merge dataframes using an outer join
merge1 = pd.merge(renamed_avg_scores, new_num_pass_math, on="school_name", how="outer")
merge2 = pd.merge(merge1, new_num_pass_read, on="school_name", how="outer")
merge3 = pd.merge(merge2, new_num_pass_both, on="school_name", how="outer")
merge4 = pd.merge(merge3, school_data, on="school_name", how="outer")
merge4

#Add Per Student Budget and Percent Passing
merge4["Per Student Budget"] = merge4["budget"]/merge4["Total Students"]
merge4["% Passing Math"] = (merge4["math_score"]/merge4["Total Students"]) * 100
merge4["% Passing Reading"] = (merge4["reading_score"]/merge4["Total Students"]) * 100
merge4["% Passing Overall"] = (merge4["size_x"]/merge4["Total Students"]) * 100
merge4.head()

# Drop unnecessary columns
drop1 = merge4.drop('School ID', 1)
drop2 = drop1.drop('size_y', 1)
drop3 = drop2.drop('size_x', 1)
drop4 = drop3.drop('math_score', 1)
school_summary = drop4.drop('reading_score', 1)
school_summary.head()

# Reorganize Columns
organized_school_summary = school_summary[["school_name","type","Total Students","budget","Per Student Budget","Math Average","Reading Average","% Passing Math","% Passing Reading", "% Passing Overall"]]
renamed_school_summary = organized_school_summary.rename(columns={"school_name":"School Name", "Reading Average":"Average Reading Score", "Math Average":"Average Math Score", "type":"Type", "budget":"Total Budget"})
renamed_school_summary


,School Name,Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,Bailey High School,District,4976.0,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858.0,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949.0,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,2739.0,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468.0,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635.0,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,427.0,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,2917.0,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,4761.0,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962.0,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [16]:
#Sort schools from highest % passing overall to lowest
top_schools = renamed_school_summary.sort_values("% Passing Overall", ascending=False)

# only take the top five
top_five_schools = top_schools.head()
top_five_schools

# Reset Index
new_index_top_five = top_five_schools.reset_index(drop=True)
new_index_top_five

,School Name,Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,Cabrera High School,Charter,1858.0,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
1,Thomas High School,Charter,1635.0,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
2,Griffin High School,Charter,1468.0,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
3,Wilson High School,Charter,2283.0,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
4,Pena High School,Charter,962.0,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [17]:
#Sort schools from lowest % passing overall to highest
low_schools = renamed_school_summary.sort_values("% Passing Overall", ascending=True)

# only take the bottom five
low_five_schools = low_schools.head()
low_five_schools

# Reset Index
new_index_low_five = low_five_schools.reset_index(drop=True)
new_index_low_five

,School Name,Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,Rodriguez High School,District,3999.0,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
1,Figueroa High School,District,2949.0,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
2,Huang High School,District,2917.0,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
3,Hernandez High School,District,4635.0,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
4,Johnson High School,District,4761.0,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [18]:
# Extract columns for grade level, school name, and math score
math_and_grades = school_data_complete[["school_name", "math_score", "grade"]]

#loc 9th Grade
frosh_math_scores = math_and_grades.loc[(math_and_grades['grade'] == "9th"), :]

#group by school and calculate average scores
grouped_frosh_math_scores = frosh_math_scores.groupby(['school_name'])

avg = grouped_frosh_math_scores.mean()

# rename columns
frosh_math_avg = avg.rename(columns={"math_score":"9th Grade Avg Math Score"})

#loc 10th grade
soph_math_scores = math_and_grades.loc[(math_and_grades['grade'] == "10th"), :]

#group by school and calculate average scores
grouped_soph_math_scores = soph_math_scores.groupby(['school_name'])

avg = grouped_soph_math_scores.mean()

# rename columns
soph_math_avg = avg.rename(columns={"math_score":"10th Grade Avg Math Score"})

#loc 11th grade
jun_math_scores = math_and_grades.loc[(math_and_grades['grade'] == "11th"), :]

#group by school and calculate average scores
grouped_jun_math_scores = jun_math_scores.groupby(['school_name'])

avg = grouped_jun_math_scores.mean()

# rename columns
jun_math_avg = avg.rename(columns={"math_score":"11th Grade Avg Math Score"})

#loc 12th grade
sen_math_scores = math_and_grades.loc[(math_and_grades['grade'] == "12th"), :]

#group by school and calculate average scores
grouped_sen_math_scores = sen_math_scores.groupby(['school_name'])

avg = grouped_sen_math_scores.mean()

# rename columns
sen_math_avg = avg.rename(columns={"math_score":"12th Grade Avg Math Score"})

# merge all the grades
merge1 = pd.merge(frosh_math_avg, soph_math_avg, on="school_name", how="outer")
merge2 = pd.merge(merge1, jun_math_avg, on="school_name", how="outer")
merge3 = pd.merge(merge2, sen_math_avg, on="school_name", how="outer")

# rename column
math_avg_by_grade = merge3.rename(columns={"school_name":"School Name"})
math_avg_by_grade

,9th Grade Avg Math Score,10th Grade Avg Math Score,11th Grade Avg Math Score,12th Grade Avg Math Score
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [19]:
# Extract columns for grade level, school name, and reading score
read_and_grades = school_data_complete[["school_name", "reading_score", "grade"]]

#loc 9th Grade
frosh_read_scores = read_and_grades.loc[(read_and_grades['grade'] == "9th"), :]

#group by school and calculate average scores
grouped_frosh_read_scores = frosh_read_scores.groupby(['school_name'])

avg = grouped_frosh_read_scores.mean()

# rename columns
frosh_read_avg = avg.rename(columns={"reading_score":"9th Grade Avg Reading Score"})

#loc 10th grade
soph_read_scores = read_and_grades.loc[(read_and_grades['grade'] == "10th"), :]

#group by school and calculate average scores
grouped_soph_read_scores = soph_read_scores.groupby(['school_name'])

avg = grouped_soph_read_scores.mean()

# rename columns
soph_read_avg = avg.rename(columns={"reading_score":"10th Grade Avg Reading Score"})

#loc 11th grade
jun_read_scores = read_and_grades.loc[(read_and_grades['grade'] == "11th"), :]

#group by school and calculate average scores
grouped_jun_read_scores = jun_read_scores.groupby(['school_name'])

avg = grouped_jun_read_scores.mean()

# rename columns
jun_read_avg = avg.rename(columns={"reading_score":"11th Grade Avg Reading Score"})

#loc 12th grade
sen_read_scores = read_and_grades.loc[(read_and_grades['grade'] == "12th"), :]

#group by school and calculate average scores
grouped_sen_read_scores = sen_read_scores.groupby(['school_name'])

avg = grouped_sen_read_scores.mean()

# rename columns
sen_read_avg = avg.rename(columns={"reading_score":"12th Grade Avg Reading Score"})

# merge all the grades
merge1 = pd.merge(frosh_read_avg, soph_read_avg, on="school_name", how="outer")
merge2 = pd.merge(merge1, jun_read_avg, on="school_name", how="outer")
merge3 = pd.merge(merge2, sen_read_avg, on="school_name", how="outer")

# rename column
read_avg_by_grade = merge3.rename(columns={"school_name":"School Name"})
read_avg_by_grade

,9th Grade Avg Reading Score,10th Grade Avg Reading Score,11th Grade Avg Reading Score,12th Grade Avg Reading Score
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [20]:
# Create the bins in which Data will be held  
bins = [0, 599, 624, 649, 1000000]

# Create the names for the five bins
group_names = ["<$600/student", "$600-624/student", "$625-649/student", ">$650/student",]

renamed_school_summary["Spending per Student"] = pd.cut(renamed_school_summary["Per Student Budget"], bins, labels=group_names, include_lowest=True)

#extract necessary columns
budget_p_student_df = renamed_school_summary[["Spending per Student", "Average Math Score", "Average Reading Score","% Passing Math","% Passing Reading","% Passing Overall"]]

#group by Spending per Student categories
grouped_budget_p_student_df = budget_p_student_df.groupby(['Spending per Student'])

#Calculate average scores
budget_p_student_scores = grouped_budget_p_student_df.mean()
budget_p_student_scores 

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending per Student,,,,,
<$600/student,83.455399,83.933814,93.460096,96.610877,90.369459
$600-624/student,83.599686,83.885211,94.230858,95.900287,90.216324
$625-649/student,79.079225,81.891436,75.668212,86.106569,66.112060
>$650/student,76.997210,81.027843,66.164813,81.133951,53.526855


In [21]:
# Create the bins in which Data will be held  
bins2 = [0, 999, 2499, 4000, 1000000]

# Create the names for the five bins
group_names2 = ["<1000", "1000-2499", "2500-4000", ">4000"]

renamed_school_summary["School Size"] = pd.cut(renamed_school_summary["Total Students"], bins2, labels=group_names2, include_lowest=True)

#extract necessary columns
student_pop_df = renamed_school_summary[["School Size", "Average Math Score", "Average Reading Score","% Passing Math","% Passing Reading","% Passing Overall"]]

#group by Spending per Student categories
grouped_student_pop_df = student_pop_df.groupby(['School Size'])

#Calculate average scores
student_pop_scores = grouped_student_pop_df.mean()
student_pop_scores

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
<1000,83.821598,83.929843,93.550225,96.099437,89.883853
1000-2499,83.357937,83.885280,93.644365,96.748840,90.615040
2500-4000,76.821621,80.957921,66.587147,80.393681,53.499124
>4000,77.136883,80.978256,66.496861,81.339570,53.902988


In [22]:
#extract necessary columns
student_type_df = renamed_school_summary[["Type", "Average Math Score", "Average Reading Score","% Passing Math","% Passing Reading","% Passing Overall"]]

#group by Spending per Student categories
grouped_student_type_df = student_type_df.groupby(['Type'])

#Calculate average scores
student_type_scores = grouped_student_type_df.mean()
student_type_scores

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
